In [1]:
import json
import pandas as pd
import C2Analyse
import C2Analyse as C2A
import matplotlib.pyplot as plt
import importlib
import re
import missingno as msno
import plotly.express as px
import numpy as np
from datetime import datetime
importlib.reload(C2A)
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

pd.options.display.max_columns = None
reload_data = 0
verbose = 1
path_folder = "./output_2021-2019/"

clean = C2A.Clean(verbose=verbose)

def percent_complete(df):
    print(df.notnull().mean()*100)

In [2]:
if reload_data == 1:
    clean.load_JSON(path_folder=path_folder)

In [3]:
#reload from CSV (faster)
clean.load_csv(path_folder=path_folder)

if clean.verbose == 1:
    display(clean.df.athletes)
    display(clean.df.extended)
    display(clean.df.workouts)

/home/adam/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3427: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/adam/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3427: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,profile_id,name,availablity,age,country,location,team,height,weight,logbook id,member since,retrieved,affiliation,wingspan,website,email,adaptive rowing category,adaptive skiing category
0,1157105,Benjamin Smith,available,32,Australia,"Brisbane, QLD",Fusion Sport,6 ft 0 in,162 lb,1157105.0,"September 28, 2017",13-03-2021 12:24:37,NaN,NaN,NaN,NaN,NaN,NaN
1,1120032,Jasper Liu,available,26,United States,NaN,NaN,NaN,NaN,1120032.0,"December 30, 2016",13-03-2021 12:24:37,NaN,NaN,NaN,NaN,NaN,NaN
2,451524,Chris Gray,available,29,United States,"Raleigh, NC",Row House Raleigh Midtown,6 ft 4 in,190 lb,451524.0,"November 22, 2007",13-03-2021 12:24:38,University of North Carolina at Chapel Hill,NaN,NaN,NaN,NaN,NaN
3,1301914,Frederick Hancock,available,19,United Kingdom,NaN,Forum Flyers (since 2001),6 ft 2 in,163 lb,1301914.0,"December 04, 2019",13-03-2021 12:24:38,Forum Flyers,6 ft 5 in,NaN,NaN,NaN,NaN
4,1189208,Itai Almogy,available,19,United States,NaN,NaN,NaN,NaN,1189208.0,"March 08, 2018",13-03-2021 12:24:38,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81022,1250363,John Oney,available,23,United States,"Greenwich, OH",NaN,5 ft 10 in,177 lb,1250363.0,"February 08, 2019",14-03-2021 14:03:20,NaN,NaN,NaN,NaN,NaN,NaN
81023,737313,Alan Aikens,available,54,United States,"Dorchester, MA",Cambridge Boat Club,NaN,238 lb,737313.0,"September 28, 2008",14-03-2021 14:03:20,Cambridge Boat Club,NaN,NaN,NaN,NaN,NaN
81024,1210804,George Watts,available,41,United States,UT,NaN,5 ft 7 in,178 lb,1210804.0,"August 18, 2018",14-03-2021 14:03:23,NaN,NaN,NaN,NaN,NaN,NaN
81025,1083487,Amir Mishriky,available,43,Egypt,Cairo,Fitness Matters Indoor Rowing,5 ft 10 in,185 lb,1083487.0,"February 21, 2016",14-03-2021 14:03:29,Fitness Matters,NaN,NaN,NaN,NaN,NaN


,workout_id,age,gender,weight class,country,verified,type,time,distance,pace,date,entered,retrieved,race,adaptive category
0,45106891,18.0,Male,Lwt,United States,Yes,Indoor Rower,1:00.0,379m,1:19.1,"June 13, 2020",Web,13-03-2021 12:24:37,NaN,NaN
1,51182039,42.0,Male,Lwt,United States,Yes,Indoor Rower,1:00.0,366m,1:21.9,"February 02, 2021 18:15:00",ErgData iOS,13-03-2021 12:24:38,NaN,NaN
2,51157298,32.0,Male,Lwt,Australia,Yes,Indoor Rower,1:00.0,397m,1:15.5,"February 02, 2021 20:16:00",ErgData Android,13-03-2021 12:24:38,NaN,NaN
3,50803540,24.0,Male,Lwt,France,Yes,Indoor Rower,1:00.0,366m,1:21.9,"January 23, 2021 23:08:00",ErgData Android,13-03-2021 12:24:39,NaN,NaN
4,49949686,19.0,Male,Lwt,United Kingdom,Yes,Indoor Rower,1:00.0,389m,1:17.1,"January 02, 2021 19:28:00",ErgData iOS,13-03-2021 12:24:39,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331742,34619955,55.0,Female,NaN,United Kingdom,No,BikeErg,8:22.0,"4,000m",2:05.5,"September 07, 2018 18:16:00",ErgData iOS,14-03-2021 14:03:42,NaN,NaN
331743,34898628,52.0,Female,NaN,Norway,No,BikeErg,8:23.8,"4,000m",2:05.9,"September 30, 2018",Web,14-03-2021 14:03:42,NaN,NaN
331744,35443299,22.0,Female,NaN,United States,No,BikeErg,8:24.9,"4,000m",2:06.2,"November 15, 2018",Web,14-03-2021 14:03:42,NaN,NaN
331745,34790536,27.0,Female,NaN,United States,Yes,BikeErg,8:25.6,"4,000m",2:06.4,"September 22, 2018 13:09:00",ErgData Android,14-03-2021 14:03:42,NaN,NaN


,workout_id,pos.,name,age,location,country,club/affiliation,distance,verified,year,machine,event,retrieved,profile_id,rower,weight,gender,adaptive,category,time
0,51157298,1,Benjamin Smith,32,"Brisbane, QLD",AUS,NaN,397.0,Yes,2021,rower,1,13-03-2021 12:24:35,1157105.0,rower,L,M,NaN,NaN,NaN
1,49949686,2,Frederick Hancock,19,NaN,GBR,Forum Flyers,389.0,Yes,2021,rower,1,13-03-2021 12:24:35,1301914.0,rower,L,M,NaN,NaN,NaN
2,48271042,3,Jasper Liu,26,NaN,USA,NaN,386.0,Yes,2021,rower,1,13-03-2021 12:24:35,1120032.0,rower,L,M,NaN,NaN,NaN
3,45106891,4,Itai Almogy,18,NaN,USA,NaN,379.0,Yes,2021,rower,1,13-03-2021 12:24:35,1189208.0,rower,L,M,NaN,NaN,NaN
4,48223141,5,Chris Gray,29,"Raleigh, NC",USA,University of North Carolina at Chapel Hill,372.0,Yes,2021,rower,1,13-03-2021 12:24:35,451524.0,rower,L,M,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331939,38060086,42,Steven Meador,63,"Hershey 🇺🇸, PA",USA,Washington University in St. Louis,NaN,Yes,2019,bikeerg,100000,14-03-2021 04:06:48,1060250.0,NaN,NaN,M,NaN,NaN,4:26:36.6
331940,36865147,1,Kirsten Kline,30,"Houston, TX",USA,Engine by Carson (EBC),NaN,Yes,2019,bikeerg,100000,14-03-2021 04:06:50,1223199.0,NaN,NaN,F,NaN,NaN,3:20:23.4
331941,34894246,2,Katrin Jüch,42,Pattensen,GER,Mobility3sixty,NaN,Yes,2019,bikeerg,100000,14-03-2021 04:06:50,1189710.0,NaN,NaN,F,NaN,NaN,4:11:31.8
331942,33478889,3,Terry Laskey,64,"Morrisville, VT",USA,"Concept2, Inc, Morrisville, VT",NaN,Yes,2019,bikeerg,100000,14-03-2021 04:06:50,7270.0,NaN,NaN,F,NaN,NaN,4:33:07.0


# Workouts

Sort and rename columns for easier working

In [4]:
#rename some columns
rename_columns = {"pos.":"pos", "club/affiliation":"affil", "category":"adaptive_cat", "weight":"weight_class"}
clean.df.workouts = clean.df.workouts.rename(columns=rename_columns)

#drop some columns
columns_to_drop = ["retrieved", "name"]
# wingspan dropped, not enough data to be useful
clean.df.workouts = clean.df.workouts.drop(columns_to_drop, axis=1)

#sort columns alphabetically
clean.df.workouts.sort_index(axis=1,inplace=True)


## Missing Data

### Time & Distance

Fixed distance events have no distance, fixed time events have no time, can fill this in from the event.

In [5]:
event_to_timedistance = {
    1:"1:00.0",
    4:"4:00.0",
    30:"30:00.0",
    60:"60:00.0",
    100:"100",
    200:"200",
    500:"500",
    1000:"1000",
    2000:"2000",
    4000:"4000",
    5000:"5000",
    6000:"6000",
    10000:"10000",
    20000:"20000",
    21097:"21097",
    40000:"40000",
    42195:"42195",
    100000:"100000"
}

clean.df.workouts.loc[clean.df.workouts["time"].isnull(), ["time"]] = clean.df.workouts[clean.df.workouts["time"].isnull()]["event"].replace(event_to_timedistance)
clean.df.workouts.loc[clean.df.workouts["distance"].isnull(), ["distance"]] = clean.df.workouts[clean.df.workouts["distance"].isnull()]["event"].replace(event_to_timedistance)

In [6]:
percent_complete(clean.df.workouts)

adaptive          0.791399
adaptive_cat      0.791399
affil            35.945521
age             100.000000
country          99.999397
distance        100.000000
event           100.000000
gender          100.000000
location         67.188743
machine         100.000000
pos             100.000000
profile_id       92.859940
rower            90.226363
time            100.000000
verified        100.000000
weight_class     90.226363
workout_id      100.000000
year            100.000000
dtype: float64


### Rower/Machine

Only need one of these columns, will keep machine.

If machine is "rower", replace with the value from "rower", otherwise leave alone.


In [7]:
clean.df.workouts.loc[clean.df.workouts["machine"]=="rower",["machine"]] = clean.df.workouts[clean.df.workouts["machine"]=="rower"]["rower"]
clean.df.workouts = clean.df.workouts.drop("rower", axis=1)

In [8]:
percent_complete(clean.df.workouts)

adaptive          0.791399
adaptive_cat      0.791399
affil            35.945521
age             100.000000
country          99.999397
distance        100.000000
event           100.000000
gender          100.000000
location         67.188743
machine         100.000000
pos             100.000000
profile_id       92.859940
time            100.000000
verified        100.000000
weight_class     90.226363
workout_id      100.000000
year            100.000000
dtype: float64


### Adaptive

Adaptive_cat contains more detail than adaptive, drop adaptive. Change nan to "able"

In [9]:
clean.df.workouts = clean.df.workouts.drop("adaptive", axis=1)
clean.df.workouts.loc[:, ["adaptive_cat"]] = clean.df.workouts["adaptive_cat"].replace({np.nan:"able"})

In [10]:
clean.df.workouts["adaptive_cat"].value_counts()

able         329317
PR3 (PD)        832
PR3             450
PR3 (VI)        265
PR3 (ID)        242
PR2             232
PR1             152
STAL             75
SIT3             65
PR3 (SA)         49
PR3 (AK)         48
SIT2             47
STA2             38
SIT1             35
STAN             26
FES              25
PR3 (GD)         20
VI               10
PR3 (AKD)         9
STA1              7
Name: adaptive_cat, dtype: int64

In [11]:
percent_complete(clean.df.workouts)

adaptive_cat    100.000000
affil            35.945521
age             100.000000
country          99.999397
distance        100.000000
event           100.000000
gender          100.000000
location         67.188743
machine         100.000000
pos             100.000000
profile_id       92.859940
time            100.000000
verified        100.000000
weight_class     90.226363
workout_id      100.000000
year            100.000000
dtype: float64


### Weight Class

Skierg and bikeerg have no weight classes, turn missing vals into H

In [12]:
clean.df.workouts.loc[:, ["weight_class"]] = clean.df.workouts["weight_class"].replace({np.nan:"H"})

In [13]:
percent_complete(clean.df.workouts)

adaptive_cat    100.000000
affil            35.945521
age             100.000000
country          99.999397
distance        100.000000
event           100.000000
gender          100.000000
location         67.188743
machine         100.000000
pos             100.000000
profile_id       92.859940
time            100.000000
verified        100.000000
weight_class    100.000000
workout_id      100.000000
year            100.000000
dtype: float64


### Country

Not many, assume all missing are USA

In [14]:
clean.df.workouts.loc[:, ["country"]] = clean.df.workouts["country"].replace({np.nan:"USA"})

In [15]:
percent_complete(clean.df.workouts)

adaptive_cat    100.000000
affil            35.945521
age             100.000000
country         100.000000
distance        100.000000
event           100.000000
gender          100.000000
location         67.188743
machine         100.000000
pos             100.000000
profile_id       92.859940
time            100.000000
verified        100.000000
weight_class    100.000000
workout_id      100.000000
year            100.000000
dtype: float64


### Affil

For those with no affilation, we will give them a value of "sol"

In [16]:
clean.df.workouts.loc[:, ["affil"]] = clean.df.workouts["affil"].replace({np.nan:"solo"})

In [17]:
percent_complete(clean.df.workouts)

adaptive_cat    100.000000
affil           100.000000
age             100.000000
country         100.000000
distance        100.000000
event           100.000000
gender          100.000000
location         67.188743
machine         100.000000
pos             100.000000
profile_id       92.859940
time            100.000000
verified        100.000000
weight_class    100.000000
workout_id      100.000000
year            100.000000
dtype: float64


### Location

Too much missing to be useful, no good way to impute date, drop the whole column.

In [18]:
clean.df.workouts = clean.df.workouts.drop("location", axis=1)

In [19]:
percent_complete(clean.df.workouts)

adaptive_cat    100.00000
affil           100.00000
age             100.00000
country         100.00000
distance        100.00000
event           100.00000
gender          100.00000
machine         100.00000
pos             100.00000
profile_id       92.85994
time            100.00000
verified        100.00000
weight_class    100.00000
workout_id      100.00000
year            100.00000
dtype: float64


### Profile ID

Without a profile ID, can't link to other workouts, drop these.

In [20]:
clean.df.workouts = clean.df.workouts[clean.df.workouts["profile_id"].notnull()]

In [21]:
percent_complete(clean.df.workouts)

adaptive_cat    100.0
affil           100.0
age             100.0
country         100.0
distance        100.0
event           100.0
gender          100.0
machine         100.0
pos             100.0
profile_id      100.0
time            100.0
verified        100.0
weight_class    100.0
workout_id      100.0
year            100.0
dtype: float64


In [22]:
clean.df.workouts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308243 entries, 0 to 331943
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   adaptive_cat  308243 non-null  object 
 1   affil         308243 non-null  object 
 2   age           308243 non-null  int64  
 3   country       308243 non-null  object 
 4   distance      308243 non-null  object 
 5   event         308243 non-null  int64  
 6   gender        308243 non-null  object 
 7   machine       308243 non-null  object 
 8   pos           308243 non-null  int64  
 9   profile_id    308243 non-null  float64
 10  time          308243 non-null  object 
 11  verified      308243 non-null  object 
 12  weight_class  308243 non-null  object 
 13  workout_id    308243 non-null  int64  
 14  year          308243 non-null  int64  
dtypes: float64(1), int64(5), object(9)
memory usage: 37.6+ MB


### Workout Time to Seconds

In [23]:
clean.df.workouts["time"] = clean.df.workouts["time"].apply(C2A.duration_string_to_duration_seconds)

### Data Types

In [24]:
datatypes = {
    "adaptive_cat":"category",
    "affil":str,
    "age":int,
    "country":"category",
    "distance":int,
    "event":"category",
    "gender": "category",
    "machine":"category",
    "pos":int,
    "profile_id":int,
    "time":float,
    "verified":"category",
    "weight_class":"category",
    "workout_id":int,
    "year":int
}
   
clean.df.workouts  = clean.df.workouts.astype(datatypes)

In [25]:
clean.df.workouts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308243 entries, 0 to 331943
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   adaptive_cat  308243 non-null  category
 1   affil         308243 non-null  object  
 2   age           308243 non-null  int64   
 3   country       308243 non-null  category
 4   distance      308243 non-null  int64   
 5   event         308243 non-null  category
 6   gender        308243 non-null  category
 7   machine       308243 non-null  category
 8   pos           308243 non-null  int64   
 9   profile_id    308243 non-null  int64   
 10  time          308243 non-null  float64 
 11  verified      308243 non-null  category
 12  weight_class  308243 non-null  category
 13  workout_id    308243 non-null  int64   
 14  year          308243 non-null  int64   
dtypes: category(7), float64(1), int64(6), object(1)
memory usage: 23.5+ MB


# Athletes

Sort and rename columns for easier working

In [26]:
clean.df.athletes = clean.df.athletes.reset_index()

#sort columns alphabetically
clean.df.athletes.sort_index(axis=1,inplace=True)

In [27]:
clean.df.athletes

,adaptive rowing category,adaptive skiing category,affiliation,age,availablity,country,email,height,index,location,logbook id,member since,name,profile_id,retrieved,team,website,weight,wingspan
0,NaN,NaN,NaN,32,available,Australia,NaN,6 ft 0 in,0,"Brisbane, QLD",1157105.0,"September 28, 2017",Benjamin Smith,1157105,13-03-2021 12:24:37,Fusion Sport,NaN,162 lb,NaN
1,NaN,NaN,NaN,26,available,United States,NaN,NaN,1,NaN,1120032.0,"December 30, 2016",Jasper Liu,1120032,13-03-2021 12:24:37,NaN,NaN,NaN,NaN
2,NaN,NaN,University of North Carolina at Chapel Hill,29,available,United States,NaN,6 ft 4 in,2,"Raleigh, NC",451524.0,"November 22, 2007",Chris Gray,451524,13-03-2021 12:24:38,Row House Raleigh Midtown,NaN,190 lb,NaN
3,NaN,NaN,Forum Flyers,19,available,United Kingdom,NaN,6 ft 2 in,3,NaN,1301914.0,"December 04, 2019",Frederick Hancock,1301914,13-03-2021 12:24:38,Forum Flyers (since 2001),NaN,163 lb,6 ft 5 in
4,NaN,NaN,NaN,19,available,United States,NaN,NaN,4,NaN,1189208.0,"March 08, 2018",Itai Almogy,1189208,13-03-2021 12:24:38,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81022,NaN,NaN,NaN,23,available,United States,NaN,5 ft 10 in,81022,"Greenwich, OH",1250363.0,"February 08, 2019",John Oney,1250363,14-03-2021 14:03:20,NaN,NaN,177 lb,NaN
81023,NaN,NaN,Cambridge Boat Club,54,available,United States,NaN,NaN,81023,"Dorchester, MA",737313.0,"September 28, 2008",Alan Aikens,737313,14-03-2021 14:03:20,Cambridge Boat Club,NaN,238 lb,NaN
81024,NaN,NaN,NaN,41,available,United States,NaN,5 ft 7 in,81024,UT,1210804.0,"August 18, 2018",George Watts,1210804,14-03-2021 14:03:23,NaN,NaN,178 lb,NaN
81025,NaN,NaN,Fitness Matters,43,available,Egypt,NaN,5 ft 10 in,81025,Cairo,1083487.0,"February 21, 2016",Amir Mishriky,1083487,14-03-2021 14:03:29,Fitness Matters Indoor Rowing,NaN,185 lb,NaN


# Drop Columns

Drop some useless columns

In [28]:
columns_to_drop = ["location", "index", "website", "email", "wingspan", "team", "availablity", "retrieved", "name"]
# wingspan dropped, not enough data to be useful
clean.df.athletes = clean.df.athletes.drop(columns_to_drop, axis=1)

## Adaptive, Affiliation

nan to able/solo

In [29]:
clean.df.athletes.loc[:, ["adaptive rowing category"]] = clean.df.athletes["adaptive rowing category"].replace({np.nan:"able"})
clean.df.athletes.loc[:, ["adaptive skiing category"]] = clean.df.athletes["adaptive skiing category"].replace({np.nan:"able"})
clean.df.athletes.loc[:, ["affiliation"]] = clean.df.athletes["affiliation"].replace({np.nan:"solo"})

In [30]:
percent_complete(clean.df.athletes)

adaptive rowing category    100.000000
adaptive skiing category    100.000000
affiliation                 100.000000
age                          92.675281
country                      92.675281
height                       45.768694
logbook id                   92.675281
member since                 92.675281
profile_id                  100.000000
weight                       50.560924
dtype: float64


## Logbook id

A missing logbook ID also indicates we are missing age, weight and other key paramters, drop all records with missing logbook id

In [31]:
clean.df.athletes = clean.df.athletes[clean.df.athletes["logbook id"].notnull()]

In [32]:
percent_complete(clean.df.athletes)

adaptive rowing category    100.000000
adaptive skiing category    100.000000
affiliation                 100.000000
age                         100.000000
country                     100.000000
height                       49.386086
logbook id                  100.000000
member since                100.000000
profile_id                  100.000000
weight                       54.557077
dtype: float64


### Height and Weight

Convert to metric, unitless values, convert/remove impossible values.

Some values missing, will impute later

In [33]:
clean.df.athletes.loc[:, ["height"]] = C2A.convert_heights(clean.df.athletes["height"])
clean.df.athletes.loc[:, ["height"]] = clean.df.athletes["height"].apply(C2A.clean_heights)

In [34]:
percent_complete(clean.df.athletes)

adaptive rowing category    100.000000
adaptive skiing category    100.000000
affiliation                 100.000000
age                         100.000000
country                     100.000000
height                       49.234273
logbook id                  100.000000
member since                100.000000
profile_id                  100.000000
weight                       54.557077
dtype: float64


In [35]:
clean.df.athletes.loc[:, ["weight"]] = C2A.convert_weights(clean.df.athletes["weight"])

0        162
1        NaN
2        190
3        163
4        NaN
        ... 
81022    177
81023    238
81024    178
81025    185
81026    201
Name: weight, Length: 75092, dtype: object


In [36]:
weight_threshold = 300
clean.df.athletes.loc[:,["weight"]] = clean.df.athletes["weight"].map(lambda x: np.nan if (x > weight_threshold) else x)

In [37]:
percent_complete(clean.df.athletes)

adaptive rowing category    100.000000
adaptive skiing category    100.000000
affiliation                 100.000000
age                         100.000000
country                     100.000000
height                       49.234273
logbook id                  100.000000
member since                100.000000
profile_id                  100.000000
weight                       54.537101
dtype: float64


### Dates

Convert all date string to datetime

In [38]:
col_list = ["member since"]

for col in col_list:
    clean.df.athletes[col] = clean.df.athletes[col].apply(C2A.convert_to_datetime)

### Age

Some ages not present ("-"), small amount, drop from dataset

In [39]:
clean.df.athletes.loc[:,["age"]] = clean.df.athletes["age"].replace("-", np.nan)
clean.df.athletes = clean.df.athletes[clean.df.athletes["age"].notnull()]
percent_complete(clean.df.athletes)

adaptive rowing category    100.000000
adaptive skiing category    100.000000
affiliation                 100.000000
age                         100.000000
country                     100.000000
height                       49.200970
logbook id                  100.000000
member since                100.000000
profile_id                  100.000000
weight                       54.543019
dtype: float64


### Data Types

Change Data Types

In [40]:
datatypes = {
    "adaptive rowing category":"category",
    "adaptive skiing category":"category",
    "affiliation":str,
    "age":int,
    "country":"category",
    "height":float,
    "logbook id":str,
    "member since": "datetime64",
    "profile_id":int,
    "weight":float
}
   
clean.df.athletes  = clean.df.athletes.astype(datatypes)

In [41]:
clean.df.athletes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74653 entries, 0 to 81026
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   adaptive rowing category  74653 non-null  category      
 1   adaptive skiing category  74653 non-null  category      
 2   affiliation               74653 non-null  object        
 3   age                       74653 non-null  int64         
 4   country                   74653 non-null  category      
 5   height                    36730 non-null  float64       
 6   logbook id                74653 non-null  object        
 7   member since              74653 non-null  datetime64[ns]
 8   profile_id                74653 non-null  int64         
 9   weight                    40718 non-null  float64       
dtypes: category(3), datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 4.8+ MB


In [76]:
clean.df.athletes.to_csv("analysis/athletes_clean.csv")
clean.df.workouts.to_csv("analysis/workouts_clean.csv")